In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 0.5
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 1

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 13.30epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.49epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 13.93epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.37epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei        dp          eo        eodd
---------------  ---------  ----------------  --------------  --------  --------  ----------  ----------
         0.0001          0          0.969453        0.979688  0.352034  0.128651  0           0.044675
         0.001           0          0.992188        0.9925    0.105635  0.10718   0.00796768  0.00796768
         0.01            0          0.998281        0.999062  0.101969  0.110085  0.00157109  0.00157109
         0.1             0          0.998125        0.998437  0.100042  0.109446  0.00314218  0.00314218


The learning rate is decided as 0.01.

In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0

#### EI Hyperparameter Selection

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.5, 0.7, 0.8, 0.85]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   1%|          | 1/100 [00:00<00:16,  6.05epochs/s]/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Training: 100%|██████████| 100/100 [00:12<00:00,  7.96epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.56epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.63epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.74epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo        eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ----------
           0.01       0             0.998281        0.999062  0.102734   0.110085  0.00157109  0.00157109
           0.01       0.2           0.998359        0.998437  0.090925   0.109446  0.00314218  0.00314218
           0.01       0.5           0.9975          0.998437  0.0623557  0.108807  0.00392773  0.00392773
           0.01       0.7           0.996641        0.998437  0.0428556  0.108807  0.00392773  0.00392773
           0.01       0.8           0.996094        0.996875  0.0433582  0.107209  0.00785546  0.00785546
           0.01       0.85          0.602344        0.602187  0          0         0           0


The lambda for EI decided as 0.5.

In [17]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.5

#### DP Hyperparameter Selection

In [8]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.01]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 12.99epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 13.23epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 13.12epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 13.00epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ----------
           0.01       0             0.998281        0.999062  0.101969   0.110085   0.00157109  0.00157109
           0.01       0.2           0.998281        0.99875   0.102546   0.109766   0.00235664  0.00235664
           0.01       0.6           0.998203        0.999062  0.101943   0.110085   0.00157109  0.00157109
           0.01       0.8           0.998359        0.998437  0.09373    0.109446   0.00314218  0.00314218
           0.01       0.9           0.992578        0.99375   0.079683   0.104013   0.0157109   0.0157109
           0.01       0.95          0.919922        0.919687  0.0554768  0.0408028  0.00392773  0.108922


The lambda for DP is decided as 0.95.

In [9]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.95

#### EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.01]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.27epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 12.89epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 12.91epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:07<00:00, 12.86epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei        dp           eo         eodd
---------------  ---------  ----------------  --------------  --------  --------  -----------  -----------
           0.01       0             0.998281        0.999062  0.101969  0.110085  0.00157109   0.00157109
           0.01       0.2           0.998203        0.999062  0.102734  0.110085  0.00157109   0.00157109
           0.01       0.6           0.998281        0.999062  0.103745  0.110085  0.00157109   0.00157109
           0.01       0.8           0.998125        0.999375  0.11417   0.110405  0.000785546  0.000785546
           0.01       0.9           0.998125        0.999375  0.112861  0.110405  0.000785546  0.000785546
           0.01       0.95          0.997891        0.998125  0.128903  0.111683  0.000785546  0.00383203


The lambda for EO is decided as 0.8.

In [11]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.01
EO_hp['lambda_'] = 0.8

#### EODD Hyperparameter Selection

In [13]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.01]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.69epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.80epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.65epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.19epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei        dp           eo         eodd
---------------  ---------  ----------------  --------------  --------  --------  -----------  -----------
           0.01       0             0.998281        0.999062  0.101969  0.110085  0.00157109   0.00157109
           0.01       0.2           0.998203        0.999062  0.102734  0.110085  0.00157109   0.00157109
           0.01       0.6           0.998281        0.999062  0.103745  0.110085  0.00157109   0.00157109
           0.01       0.8           0.998125        0.999375  0.103664  0.110405  0.000785546  0.000785546
           0.01       0.9           0.998125        0.999375  0.112095  0.110405  0.000785546  0.000785546
           0.01       0.95          0.997969        0.99875   0.124405  0.111037  0.000561104  0.00153281


The lambda for EODD is decided as 0.8.

In [14]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.8

#### Model training

In [18]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

training seed 1 started


Training: 100%|██████████| 100/100 [00:07<00:00, 14.15epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.46epochs/s]


training seed 3 started


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.69epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.66epochs/s]

Training finished for all seeds.


In [19]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean      dp_var     eo_mean     eo_var    eodd_mean     eodd_var
-------  ---------------  --------------  ---------  ----------  ---------  ----------  ----------  ---------  -----------  -----------
SGD              0.9975      1.11022e-16  0.105857   0.00240015  0.0888411  0           0.00249377  0           0.00324295  4.33681e-19
EI               0.9485      0.0978753    0.0540348  0.0136051   0.101598   0.0286573   0.126434    0.245076    0.126434    0.245076
DP               0.92435     0.0119589    0.0790179  0.0365484   0.0154437  0.00766754  0.119327    0.0967573   0.161675    0.0486726
EO               0.9975      1.11022e-16  0.112349   0.00119981  0.0888411  0           0.00249377  0           0.00324295  4.33681e-19
EODD             0.9975      1.11022e-16  0.111949   0.00167941  0.0888411  0           0.00249377  0           0.00324295  4.33681e-19
